In [8]:
import sys, os, importlib, shutil
import rasterio, reverse_geocode

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon, box, Point

#Import raster helpers
sys.path.append("../../../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

#Import GOST urban functions
sys.path.append("../../../../GOST_Urban/")
import src.UrbanRaster as urban
import src.urban_helper as helper

In [2]:
# Define baseline data
base_folder = "/home/public/Data/PROJECTS/INFRA_SAP/COD"
national_bounds = os.path.join(base_folder, "admin.shp")
urban_extents = os.path.join(base_folder, "urban_extents.shp")

wsf_layer = "/home/public/Data/GLOBAL/WSF/Democratic Republic of the Congo/DRC_WSF_evolution.tif"

In [13]:
# Run zonal stats against wsf for admin boundaries and urban extents
inD = gpd.read_file(national_bounds)
inR = rasterio.open(wsf_layer)
exp_vals = list(range(1985,2016))
res = rMisc.zonalStats(inD, inR, rastType='C', unqVals=exp_vals)
res = pd.DataFrame(res, columns=["b_%s" % x for x in exp_vals])
res = res.cumsum(axis=1)
res['WB_ADM2_CO'] = inD['WB_ADM2_CO']
res['WB_ADM2_NA'] = inD['WB_ADM2_NA']

In [16]:
inD_urban = gpd.read_file(urban_extents)
res_urban = rMisc.zonalStats(inD_urban, inR, rastType='C', unqVals=exp_vals)
res_urban = pd.DataFrame(res_urban, columns=["b_%s" % x for x in exp_vals])
res_urban = res_urban.cumsum(axis=1)
res_urban['ID'] = inD_urban['ID']

In [17]:
centroids = [(x.centroid.y, x.centroid.x) for x in inD_urban['geometry']]
urban_names = reverse_geocode.search(centroids)
inD_urban['CITY_NAME'] = [x['city'] for x in urban_names]
inD_urban['COUNTRY'] = [x['country_code'] for x in urban_names]
res_urban['CITY_NAME'] = [x['city'] for x in urban_names]
res_urban['COUNTRY'] = [x['country_code'] for x in urban_names]

In [20]:
res.to_csv("/home/wb411133/temp/COD_ADM1_WSF_Summary.csv")

In [21]:
res_urban.to_csv("/home/wb411133/temp/COD_CITY_WSF_Summary.csv")

# Run nighttime lights 